In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [2]:
import pandas as pd
import requests as rq
import lxml as lxml
from bs4 import BeautifulSoup
import io as io
import math
import gzip
import shutil
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
from pandasgui import show
import numpy as np
from io import BytesIO
from folium.plugins import HeatMap
import nbconvert

from script import request_tri
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [21]:
# Charger le fichier CSV contenant les transactions
transactions = pd.read_csv("data/base.csv",sep=";",encoding="utf-8")

C:\Users\flori\AppData\Local\Temp\ipykernel_23056\2978799986.py:2: DtypeWarning:

Columns (13,15) have mixed types. Specify dtype option on import or set low_memory=False.



In [22]:
import pandas as pd
import time
import requests
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Fonction wrapper avec gestion des erreurs et temporisation
def check_inondable_parallel_with_retry_and_delay(args):
    latitude, longitude = args
    attempts = 2  # Nombre de tentatives en cas d'échec
    for _ in range(attempts):
        try:
            # Remplacez ici avec votre méthode d'appel API (request_tri.check_inondable)
            return request_tri.check_inondable(latitude, longitude)
        except requests.exceptions.RequestException as e:
            print(f"Erreur de connexion pour la commune (lat: {latitude}, lon: {longitude}): {e}")
            time.sleep(2)  # Délai de 2 secondes avant de réessayer
    return "Erreur de connexion"  # Retourne une valeur en cas d'échec après plusieurs tentatives

# Préparer les données nécessaires (coordonnées)
coordinates = list(zip(transactions['latitude'], transactions['longitude']))

# Fonction wrapper pour appeler check_inondable
def check_inondable_parallel(args):
    latitude, longitude = args
    return check_inondable_parallel_with_retry_and_delay(args)

# Limiter le nombre de threads simultanés pour éviter la surcharge
with ThreadPoolExecutor(max_workers=5) as executor:  # Limite de 5 threads
    results = list(executor.map(check_inondable_parallel, coordinates))

# Convertir les résultats en DataFrame
results_df = pd.DataFrame(results, columns=['results', 'identifiant_tri', 'libelle_type_inondation', 'code_scenario'])

# Fusionner les résultats avec les données originales
transactions = pd.concat([transactions.reset_index(drop=True), results_df], axis=1)

# Afficher les premiers résultats pour vérification
print(transactions.head())


Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur API (400) pour la commune 'None' (lat: nan, lon: nan).
Erreur A

In [23]:
show(transactions)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
C:\Users\flori\AppData\Local\Temp\ipykernel_23056\18237388.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_23056\18237388.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_23056\18237388.py:1: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\flori\AppData\Local\Temp\ipykernel_23056\18

In [ ]:
# Sauvegarder le fichier avec les résultats

transactions['zone_inondable'] = transactions['zone_inondable'].replace({2: 0, 3: 0})
transactions = transactions.rename(columns={'results': 'zone_inondable'})
transactions.to_csv("data/transactions_with_zone_inondable.csv",sep=";",encoding="utf-8", index=False)

In [5]:
transactions = pd.read_csv("data/transactions_with_zone_inondable.csv",sep=";",encoding="utf-8")

C:\Users\flori\AppData\Local\Temp\ipykernel_30728\1473045797.py:1: DtypeWarning: Columns (13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv("data/transactions_with_zone_inondable.csv",sep=";",encoding="utf-8")


In [6]:
# Étape 1 : Compter et afficher le nombre de transactions par commune
transaction_counts = transactions.groupby('nom_commune').size().sort_values(ascending=False)
print(transaction_counts)

nom_commune
Nice                7693
Toulon              3079
Cannes              2833
Brest               2620
Antibes             2190
                    ... 
Cléden-Cap-Sizun       1
Centuri                1
Tréduder               1
Cagnano                1
Talasani               1
Length: 562, dtype: int64


In [7]:
filtered_transactions = transactions.groupby('nom_commune').filter(
    lambda group: len(group) >= 10 and group['zone_inondable'].sum() > 5 and (group['zone_inondable'] == 0).any()
)

In [8]:
# Étape 3 : Calculer la moyenne agrégée des prix par commune et par zone inondable
filtered_transactions['prix_m2'] = filtered_transactions['valeur_fonciere'] / filtered_transactions['surface_reelle_bati']

In [9]:
# Moyenne par commune et zone inondable
mean_prices = filtered_transactions.groupby(['nom_commune', 'zone_inondable'])['prix_m2'].mean().unstack()

In [12]:
# Filtrer les transactions pour les appartements
filtered_appartements = filtered_transactions[filtered_transactions['type_local'] == 'Appartement']

# Filtrer les transactions pour les maisons
filtered_maisons = filtered_transactions[filtered_transactions['type_local'] == 'Maison']

def process_data(filtered_data, output_file):
    # Calcul du prix moyen par commune et par zone inondable
    mean_by_zone = (
        filtered_data
        .groupby(['nom_commune', 'zone_inondable'])['prix_m2']
        .mean()
        .unstack(fill_value=0)  # Remplace NaN par 0
    )
    
    # Calcul du prix moyen total par commune
    mean_total = (
        filtered_data
        .groupby('nom_commune')['prix_m2']
        .mean()
    )
    
    # Renommer les colonnes pour les prix moyens par zone
    mean_by_zone = mean_by_zone.rename(columns={0: 'prix_moyen_non_inondable', 1: 'prix_moyen_inondable'})
    
    # Ajouter la colonne pour le prix moyen total
    mean_by_zone['prix_moyen_total'] = mean_total
    
    # Calculer la différence entre les prix moyens
    mean_by_zone['différence_inondable'] = (
        mean_by_zone['prix_moyen_inondable'] - mean_by_zone['prix_moyen_non_inondable']
    )
    
    # Ajouter le nombre d'observations pour chaque zone
    observations_count = (
        filtered_data
        .groupby(['nom_commune', 'zone_inondable'])
        .size()
        .unstack(fill_value=0)  # Remplace NaN par 0
    )
    
    # Renommer les colonnes pour le comptage des observations
    observations_count = observations_count.rename(columns={0: 'nb_non_inondable', 1: 'nb_inondable'})
    
    # Joindre les observations au DataFrame 'mean_by_zone'
    mean_by_zone = mean_by_zone.join(observations_count)
    
    # Ajouter la population par commune
    population_data = filtered_data.groupby('nom_commune')['Population'].max()
    mean_by_zone = mean_by_zone.join(population_data, on='nom_commune')
    
    # Trier les communes par population décroissante et sélectionner les 20 plus peuplées
    result_table = mean_by_zone.sort_values(by='Population', ascending=False).head(20)
    
    # Exporter le DataFrame final en CSV
    result_table.to_csv(output_file, encoding='utf-8-sig', sep=";", decimal=",", index=True)
    
    return result_table

# Processus pour les appartements
print("Calcul pour les appartements...")
appart_table = process_data(filtered_appartements, 'data/moyenne_appartements.csv')
print(appart_table)

# Processus pour les maisons
print("\nCalcul pour les maisons...")
maison_table = process_data(filtered_maisons, 'data/moyenne_maisons.csv')
print(maison_table)


Calcul pour les appartements...
                       prix_moyen_non_inondable  prix_moyen_inondable  \
nom_commune                                                             
Nice                                5004.367634           4419.183377   
Toulon                              3029.709506           2312.117528   
Dunkerque                           2148.683797           2027.939264   
Cherbourg-en-Cotentin               2079.361888           2215.856493   
Calais                              1733.639657              0.000000   
Cannes                              6574.332615           7776.786163   
Antibes                             5207.041706           7032.927766   
Saint-Nazaire                       3007.717013           2378.418680   
Hyères                              3898.476035           5178.523516   
Fréjus                              4336.409823           3937.443013   
Cagnes-sur-Mer                      4668.769141           4629.627163   
Saint-Malo         